<a href="https://colab.research.google.com/github/VOL-EM2024/mo/blob/main/%D0%9B%D0%B0%D0%B1%D0%B03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import zipfile
import json
import pandas as pd
from tqdm import tqdm

In [72]:
!pip install pymorphy2
!pip install pymorphy3

In [73]:
import pymorphy3
import plotly.graph_objs as go

In [74]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [75]:
df = pd.read_csv("/content/russian_jokes.csv")

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63927 entries, 0 to 63926
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            61685 non-null  object
 1   comments_count  63927 non-null  int64 
 2   likes_count     63927 non-null  int64 
 3   reposts_count   63927 non-null  int64 
 4   views_count     63927 non-null  int64 
 5   date            63927 non-null  object
 6   unique_key      63927 non-null  object
 7   group_source    63927 non-null  object
dtypes: int64(4), object(4)
memory usage: 3.9+ MB


In [77]:
df.drop(['comments_count', 'likes_count', 'reposts_count', 'views_count', 'date'], axis=1, inplace=True)

In [78]:
df.dropna()

,text,unique_key,group_source
0,Директор отчитывает главбуха-еврея: \n- Яков С...,m3Cvn1OcrSBlJ_gABgIoRQBcyH0V,https://vk.com/jewishpugs
1,"— Изя, как вы думаете, объявления в газетах да...",GPIii1u2kEabZEbnHqLTAyZA37ES,https://vk.com/jewishpugs
2,Метро. На единственное свободное место садится...,GJLahnmtLvVcc8x41sdV-lACQNPy,https://vk.com/jewishpugs
3,Тюлька и килька вышли замуж за евреев. Теперь ...,lBev3CChZteVPLaD1GHpYawoK5a4,https://vk.com/jewishpugs
4,Судился еврей с армянином. В итоге судье дали ...,I0rV0HbWhAkGHsnPNnBaONSUkKeX,https://vk.com/jewishpugs
...,...,...,...
63922,"Просыпается мужик с дичайшего бодуна. Лежал, л...",vDjdEcFiG8YrKcjeXaSvjUijhwMb,https://vk.com/baneks
63923,-Вы кто по гороскопу?\n-Я рыба!\n-А я пиво!,ehQYA9rZywBq_iLUFL0q7Ko4lv3c,https://vk.com/baneks
63924,— Как узнать снеговик женщина или мужчина? \n—...,8fwnP1V5aLBdRMjPFe59pH7EfdWc,https://vk.com/baneks
63925,"— А у вас хлеб свежий?\n— Нет, прошлогодний!",cr-Y3pWxUJfKP4KnQ7Jl823354aC,https://vk.com/baneks


In [79]:
df.head()

,text,unique_key,group_source
0,Директор отчитывает главбуха-еврея: \n- Яков С...,m3Cvn1OcrSBlJ_gABgIoRQBcyH0V,https://vk.com/jewishpugs
1,"— Изя, как вы думаете, объявления в газетах да...",GPIii1u2kEabZEbnHqLTAyZA37ES,https://vk.com/jewishpugs
2,Метро. На единственное свободное место садится...,GJLahnmtLvVcc8x41sdV-lACQNPy,https://vk.com/jewishpugs
3,Тюлька и килька вышли замуж за евреев. Теперь ...,lBev3CChZteVPLaD1GHpYawoK5a4,https://vk.com/jewishpugs
4,Судился еврей с армянином. В итоге судье дали ...,I0rV0HbWhAkGHsnPNnBaONSUkKeX,https://vk.com/jewishpugs


In [80]:
# убираем знаки препинания и другое, числа - код в ASCII
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ',
             46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ',
             95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

In [83]:
# функция для очистки данных и создания нового столбца уже с очищенным текстом
def cleaning(dataframe):
    dataframe['Post_clean'] = dataframe['text'].map(lambda x: x.lower())
    dataframe['Post_clean'] = dataframe['Post_clean'].map(lambda x: remove_punct(x))
    dataframe['Post_clean'] = dataframe['Post_clean'].map(lambda x: x.split(' '))
    dataframe['Post_clean'] = dataframe['Post_clean'].map(lambda x: [token for token in x if token not in russian_stopwords\
                                                                  and token != " " \
                                                                  and token.strip() not in punctuation])
    dataframe['Post_clean'] = dataframe['Post_clean'].map(lambda x: ' '.join(x))

In [82]:
cleaning(df)
df.head()

AttributeError: 'float' object has no attribute 'lower'

In [66]:
text=df['Post_clean'] #берем столбец с текстом
norm_text=[]
# 1 - только буквы
# 2 - перевод в нижний регистр
# 3 - привести все слова в единую форму
morph=pymorphy2.MorphAnalyzer()
for s in tqdm(text):
    s1 = re.sub(r'[^\w\s]+|[\d]+', r'',s).strip()
    s1 = s1.lower()
    s1 = word_tokenize(s1) #токенизация
    words=[]
    for i in s1:
        pv = morph.parse(i)
        words.append(pv[0].normal_form)
    sentence=' '.join(words)
    norm_text.append(sentence)

KeyError: 'Post_clean'

In [ ]:
df['title'] = df.apply(lambda row: str(row['title']) + ' ' + str(row['genre']), axis=1)

In [ ]:
work_df = df[["title","genre","imdb_rating","plot"]]

In [ ]:
work_df = work_df.sample(n=1000)
work_df.reset_index(drop=True, inplace=True)

In [ ]:
import re

def remove_special_characters_and_numbers(text):
    pattern = re.compile(r'[^А-Яа-я\s]')
    return re.sub(pattern, '', text)

# Применяем функцию к столбцу 'plot' в DataFrame
work_df['norm_plot'] = work_df['plot'].apply(lambda x: remove_special_characters_and_numbers(str(x)))

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
work_df["norm_text"] = norm_text

In [ ]:
work_df

In [ ]:
cat23 = work_df['norm_text'].values
cat23 = ' '.join(cat23)
cat23

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.subplots_adjust(wspace=0.3, hspace=0.2)

wordcloud = WordCloud(colormap='Accent', background_color='white', contour_width=10).generate(cat23)

# Save the word cloud as an image file
wordcloud.to_file("wordcloud.png")

# Load the saved image file
wordcloud_img = plt.imread("wordcloud.png")

# Display the word cloud image
plt.imshow(wordcloud_img)
plt.axis("off")
plt.title('WordCloud', fontdict={'fontsize': 7, 'color': 'grey'}, y=0.93)
plt.show()


In [ ]:
test_text = work_df[work_df["genre"]=="криминальная драма"]['norm_text'].values
test_text = ' '.join(test_text)
test_text

In [ ]:
test_text = re.sub(r'\b\w{1,4}\b', '', test_text)

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.subplots_adjust(wspace=0.3, hspace=0.2)

wordcloud = WordCloud(colormap='Accent', background_color='white', contour_width=10).generate(test_text)
plt.tick_params(labelsize=10)
plt.imshow(wordcloud)
plt.axis("off")
plt.title('WordCloud',fontdict={'fontsize':7,'color':'grey'},y=0.93)
plt.tick_params(labelsize=10)

In [ ]:
# Подключаем синтаксической анализатор и его части.
import ufal.udpipe
from ufal.udpipe import Model, Pipeline, ProcessingError

# Для отрисовки деревьев нам потребуются вот эти библиотеки.

from collections import defaultdict
from tqdm.auto import tqdm
import random

In [ ]:
#загружаем модель
model = Model.load("russian-syntagrus-ud-2.5-191206.udpipe")

In [ ]:
# Токенизатор со снятием омонимии.
tokenizer = model.newTokenizer(model.DEFAULT)
# Объект для отображения результатов разбора.
conlluOutput = ufal.udpipe.OutputFormat.newOutputFormat("conllu")
# Объект предложения, в которое будет осуществляться разбор.
sentence = ufal.udpipe.Sentence()
# Объект для ошибок, возникающих в ходе разбора.
error = ufal.udpipe.ProcessingError()

In [ ]:
work_df

In [ ]:
work_df["plot"][0]

In [ ]:
tokenizer.setText(work_df['plot'][0])

In [ ]:
tokenizer.nextSentence(sentence, error)
model.tag(sentence, model.DEFAULT)
model.parse(sentence, model.DEFAULT)
print(conlluOutput.writeSentence(sentence))

In [ ]:
df_sentence = pd.DataFrame(columns=['head','lemma','form','upostag','feats','deprel','misc'])

In [ ]:
for i, word in enumerate(sentence.words):
  df_sentence.loc[len(df_sentence)] = [word.head, word.lemma, word.form, word.upostag, word.feats, word.deprel, word.misc]
    #print(i, "->", word.head, word.lemma, "[", word.form, "]", word.upostag, word.feats, word.deprel, word.misc)

In [ ]:
df_sentence

In [ ]:
def find_child_Ng(sentence, parent):
    """ Функция поиска всех потомков вершины с номером parent, являющихся
    существительными в родительном падеже.
    """
    children = []
    for i, word in enumerate(sentence.words):
        if word.head == parent and word.upostag == "NOUN":
            children.append(i)
    return children

def find_NNg(sentence, combinations):
    """ Функция поиска сочетаний "существительное + существительное в родительном падеже".
    """
    for i, word in enumerate(sentence.words):
        if word.upostag == "VERB":
            children = find_child_Ng(sentence, i)
            if children != []:
                #print(word.lemma, [sentence.words[k].form for k in children])
                for child in children:
                    combinations[word.lemma][sentence.words[child].form] += 1

In [ ]:
combinations = defaultdict(lambda:defaultdict(int))

for text in tqdm(work_df['plot']):
    tokenizer.setText(str(text)) # Установить текст.
    while tokenizer.nextSentence(sentence, error): # Полчаем следующее предложение.
        model.tag(sentence, model.DEFAULT) # Токенизация.
        model.parse(sentence, model.DEFAULT) # Синтаксический анализ.
        find_NNg(sentence, combinations) # Поиск сочетаний.
  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
df_nn=pd.DataFrame(columns=['parent','children','freq'])
for parent, childs in combinations.items():
    for child, freq in childs.items():
        if freq > 5:
            df_nn.loc[len(df_nn)] = [parent, child,freq]

In [ ]:
df_nn.sort_values(by='freq',ascending=False).head(20)

In [ ]:
russian_stopwords = stopwords.words("russian")
vectorizer_tfidf = TfidfVectorizer(max_features=500, min_df=20, max_df=0.7, stop_words=russian_stopwords)
text_tfidf = vectorizer_tfidf.fit_transform(norm_text)
text_tfidf = pd.DataFrame(text_tfidf.toarray(),columns=vectorizer_tfidf.get_feature_names_out())
text_tfidf.head()

In [ ]:
!pip install umap-learn

#понижение размерности
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
def show_fig(new_shape):

    fig = go.Figure(go.Scatter3d(x=new_shape[:, 0],
                        y=new_shape[:, 1],
                        z=new_shape[:, 2],
                        marker=dict(opacity=0.9,
                                    reversescale=True,
                                    colorscale='Blues',
                                    size=5),
                        line=dict (width=0.02),
                        mode='markers',
                                 text=work_df['title']))
    fig.update_layout(
        scene = dict(
            xaxis = dict(nticks=4),
                         yaxis = dict(nticks=4),
                         zaxis = dict(nticks=4),),
        width=700,
        margin=dict(r=20, l=10, b=10, t=10))
    fig.show()

In [ ]:
pca=PCA(n_components=3)
reduced_pca = pca.fit_transform(text_tfidf)
show_fig(reduced_pca)

In [ ]:
tsne=TSNE(n_components=3)
reduced_tsne = tsne.fit_transform(text_tfidf)
show_fig(reduced_tsne)

In [ ]:
import umap
umap_emb = umap.UMAP(n_components=3)
reduced_umap = umap_emb.fit_transform(text_tfidf)
show_fig(reduced_umap)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(text_tfidf)
for i,topic in enumerate(LDA.components_):
    print(f'5 слов для темы #{i}:')
    print([vectorizer_tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

In [ ]:
text_clean= []
for row in norm_text:
        text_clean.append(row.split())

from gensim.models import Phrases
bigram = Phrases(text_clean)
trigram = Phrases(bigram[text_clean])

for idx in range(len(text_clean)):
    for token in bigram[text_clean[idx]]:
        if '_' in token:

            text_clean[idx].append(token)
    for token in trigram[text_clean[idx]]:
        if '_' in token:
          text_clean[idx].append(token)

In [ ]:
from gensim.corpora.dictionary import Dictionary
from numpy import array
dictionary = Dictionary(text_clean)
dictionary.filter_extremes(no_below=10, no_above=0.1)
corpus = [dictionary.doc2bow(doc) for doc in text_clean]
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus))

In [ ]:
from gensim.models.ldamulticore import LdaMulticore
model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=2)
model.show_topics()

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim
pyLDAvis.enable_notebook()
data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(data)